In [10]:
import os
import shutil
import time

import config
config.LATEX_PDF_RESULTS.mkdir(parents=True, exist_ok=True)
# os.chdir(config.PROJECT_ROOT_PATH)

from numpy.random import choice
import random
import pathlib

import subprocess

In [11]:
def delayed(foo, args):
    def inner():
        return foo(*args)
    return inner

nl = '\n'

class PDFGenerator:
    def __init__(self, playground_path, content_path):
        self.playground_path = playground_path
        self.content_path = content_path
        self.read_words().read_formulas().read_images()
        self.buffer = []

    def read_words(self):
        with open(pathlib.Path(self.content_path).joinpath('words.txt'), 'r') as words:
            self.words = list(words.read().split())
        return self

    def read_formulas(self):
        with open(pathlib.Path(self.content_path).joinpath('formulas.txt'), 'r') as formulas:
            self.formulas = map(lambda x: x[:-1] if len(x) > 0 else '', formulas.readlines())
            # self.formulas = filter(lambda x: len(x) < 120, self.formulas)
            self.formulas = filter(lambda x: 'array' not in x, self.formulas)
            self.formulas = list(self.formulas)
            # print('AVG LEN:', sum(map(len, self.formulas))/len(self.formulas))
        return self

    def read_images(self):
        self.images_list = list(map(
            lambda x: str(pathlib.Path(self.content_path).joinpath('geometry_images_monocrome', x)),
            os.listdir(pathlib.Path(self.content_path).joinpath('geometry_images_monocrome'))
        ))
        return self

    # **************************
    # gen_smth возвращают строку
    # **************************

    def gen_sentence(self):
        return self.gen_word(upper_case=True) + ' ' + ' '.join([self.gen_word() for _ in range(random.randint(5, 15))]) + '.'

    def gen_word(self, upper_case=False):
        x = random.choice(self.words)
        if upper_case:
            x = x.title()
        return x

    def gen_header(self):
        return self.gen_word(upper_case=True) + ' ' + ' '.join([self.gen_word() for _ in range(random.randint(1, 4))])

    def gen_paragraph(self):
        return ''.join([self.gen_sentence() for _ in range(random.randint(1, 7))])

    def gen_image(self):
        return random.choice(self.images_list) + '}' + '{' + str((random.uniform(0, 0.4)**2)*1.9+0.5)

    def gen_formula(self):
        return str(random.choice(self.formulas))

    # ***********************************************
    # add_part так или иначе добавляют что-то в буфер
    # ***********************************************

    def add_geometry(self):
        left, right, top = [random.randint(5, 25), random.randint(5, 25), random.randint(5, 25)]
        self.buffer.append(
            r'\geometry{' +
            f'''
            left={left}mm,
            right={right}mm,
            top={top}mm,
            ''' +
            r'}' + nl
        )

    def add_document(self):
        random.seed(time.time())

        self.add_geometry()

        self.buffer.append(r'\begin{document}' + nl)
        self.buffer.append(r'\pagestyle{fancy}' + nl)

        self.add_chapters(chaptersnum=2)

        self.buffer.append(r'\end{document}' + nl)

        return self

    def add_part(self, part_type, content_foo):
        self.buffer.append(
            '\\' + part_type + '{' + \
            content_foo() + \
            '}\n\n'
        )

    def add_image(self):
        self.buffer.append(self.gen_image())

    # def add_formula(self):
    #     self.buffer.append(self.gen_formula())

    def add_some_paragraphs(self, a=2, b=8):
        foos = [
            delayed(
                self.add_part, ('mytext', self.gen_paragraph)
            ),
            delayed(
                self.add_part, ('smalltext', self.gen_paragraph)
            ),
            delayed(
                self.add_part, ('formula', self.gen_formula)
            ),
            delayed(
                self.add_part, ('insertimage', self.gen_image)
            ),
        ]

        parts = choice(
            a=foos,
            size=random.randint(a, b),
            p=[0.5,0.125,0.3,0.075]
        )
        for part in parts:
            part()


    def add_chapter(self):
        self.add_part('chapter', self.gen_header)
        self.add_some_paragraphs(0, 10)
        for i in range(random.randint(2,4)):
            self.add_part('mysection', self.gen_header)
            self.add_some_paragraphs(1, 4)
            self.add_part('mysubsection', self.gen_header)
            self.add_some_paragraphs(2, 13)
            self.add_part('mysubsection', self.gen_header)
            self.add_some_paragraphs(2, 13)

    def add_chapters(self, chaptersnum=3):
        for _ in range(chaptersnum):
            self.add_chapter()
        return self

    # ***********************************************
    # пишем в файл
    # ***********************************************

    def write_to_file(self):
        with open(
                self.playground_path.joinpath('content.tex'), 'w'
        ) as f:
            for x in self.buffer:
                f.write(x.replace('\_', '_').replace('_', '\_'))

        self.buffer = []

In [12]:
pdfgenerator = PDFGenerator(
    playground_path=config.LATEX_PLAYGROUND_PATH,
    content_path=config.LATEX_CONTENT_PATH
)

# pdfgenerator.add_some_paragraphs()
# print(pdfgenerator.buffer)
# pdfgenerator.add_chapters(chaptersnum=20).write_to_file()

In [13]:
os.chdir(config.LATEX_PLAYGROUND_PATH)

for i in range(15):
    pdfgenerator.add_document().write_to_file()
    proc1 = subprocess.Popen("pdflatex -synctex=1 -interaction=nonstopmode main.tex".split())
    proc2 = subprocess.Popen("pdflatex -synctex=1 -interaction=nonstopmode main_gray.tex".split())
    print('----------------------')
    output, error = proc1.communicate()
    print(output, error)
    output, error = proc2.communicate()
    print(output, error)
    shutil.copyfile('main.pdf', config.LATEX_PDF_RESULTS.joinpath(f'main{i}.pdf'))
    shutil.copyfile('main_gray.pdf', config.LATEX_PDF_RESULTS.joinpath(f'main_gray{i}.pdf'))

----------------------
This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Arch Linux) (preloaded format=pdflatex)
 restricted \write18 enabled.
This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Arch Linux) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./main_gray.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-04-10>entering extended mode
(./main.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-04-10>
(/usr/share/texmf-dist/tex/latex/base/book.cls
(/usr/share/texmf-dist/tex/latex/base/book.cls
Document Class: book 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texmf-dist/tex/latex/base/bk12.clo)
Document Class: book 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texmf-dist/tex/latex/base/bk12.clo)))
(/usr/share/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texmf-dist/tex/generic/babel/babel.sty
(/usr/share/texm

In [14]:
# %cd $LATEX_PLAYGROUND_PATH
# !pdflatex -synctex=1 -interaction=nonstopmode main.tex
# !pdflatex -synctex=1 -interaction=nonstopmode main_gray.tex
#
# %cd $PROJECT_ROOT_PATH
# shutil.copy('main.pdf', PROJECT_ROOT+'/main.pdf')